In [1]:
!make

g++ -O3 -I. nt_opt.cpp -o nt_opt
g++ -O3 -I. nt_article.cpp -o nt_article
nt_article.cpp: In function ‘uint64_t h(char)’:
nt_article.cpp:22:1: warning: control reaches end of non-void function [-Wreturn-type]
 }
 ^
nt_article.cpp: In function ‘uint64_t rc(char)’:
nt_article.cpp:39:1: warning: control reaches end of non-void function [-Wreturn-type]
 }
 ^


In [2]:
!make test

./nt_opt TGCAG
NT64C 0x0bafa6628fc6dab7
fhVal 0x0bafa6628fc6dab7
rhVal 0x8cf2d41f2cca4802
./nt_article TGCAG
NT64C 0x0bafa6728fc6dabf
fhVal 0x0bafa6728fc6dabf
rhVal 0x8cf2d4072cca480e
python nthash.py TGCAG
NTC64 0x0bafa6728fc6dabf
fhVal 0x0bafa6728fc6dabf
rhVal 0x8cf2d4072cca480e
cargo run -q TGCAG
NT64C 0x0bafa6728fc6dabf
fhVal 0x0bafa6728fc6dabf
rhVal 0x8cf2d4072cca480e


In [3]:
%%bash -l
./nt_opt AAAAA
./nt_article AAAAA
python nthash.py AAAAA
cargo run -q AAAAA

NT64C 0x01542d2a1299ba7e
fhVal 0x9b5384ab1b0279a4
rhVal 0x01542d2a1299ba7e
NT64C 0x01542d341299ba71
fhVal 0x9b5384bf1b0279ae
rhVal 0x01542d341299ba71
NTC64 0x01542d341299ba71
fhVal 0x9b5384bf1b0279ae
rhVal 0x01542d341299ba71
NT64C 0x01542d341299ba71
fhVal 0x9b5384bf1b0279ae
rhVal 0x01542d341299ba71
